In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
from typing import List, Dict
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import re

def parse_trajectories(file_path):
    with open(file_path, 'r') as f:
        content = f.read()

    blocks = content.strip().split("\n\n")  # Split by blocks separated by blank lines
    all_trajectories = []

    for block in blocks:
        lines = block.strip().split("\n")
        trajectory = []

        for line in lines:
            actions = line.strip().split("-")  # Each action is separated by '-'
            timestep = []

            for action in actions:
                # Adjust the regex to capture multiple labels in the format pX(y1,y2,...)
                match = re.match(r"r(\d+)\((\d+)\):p(\d+)\(([^)]+)\),p(\d+)\(([^)]+)\)", action.strip())
                if match:
                    r_id, duration, p_start, y_start_str, p_end, y_end_str = match.groups()
                    
                    # Extract numeric label values (e.g., 'y1', 'y7' → 1, 7)
                    y_start = [int(label[1:]) for label in y_start_str.split(',')]
                    y_end = [int(label[1:]) for label in y_end_str.split(',')]

                    timestep.append({
                        "robot_id": int(r_id),
                        "duration": int(duration),
                        "p_start": int(p_start),
                        "y_start": y_start,
                        "p_end": int(p_end),
                        "y_end": y_end
                    })
                else:
                    print(f"Warning: Could not parse action: {action}")
            trajectory.append(timestep)

        all_trajectories.append(trajectory)

    return all_trajectories

# Parse the data from the log file
parsed_data = parse_trajectories("log.txt")

# Print the parsed data for the first timestep in the first trajectory
print(parsed_data[0][0])  # first timestep in that trajectory

In [ ]:
import torch
import torch.nn as nn

# Compute max ids from the parsed data
robot_ids = set()
place_ids = set()

for traj in parsed_data:
    for timestep in traj:
        for action in timestep:
            robot_ids.add(action['robot_id'])
            place_ids.add(action['p_start'])
            place_ids.add(action['p_end'])

max_robot_id = max(robot_ids) + 1
max_place_id = max(place_ids) + 1
max_timesteps = max(len(traj) for traj in parsed_data)

# Then initialize your embeddings
embed_dim = 32
robot_embedding_matrix = nn.Embedding(max_robot_id, embed_dim)
place_embedding_matrix = nn.Embedding(max_place_id, embed_dim)
timestep_embedding_matrix = nn.Embedding(max_timesteps, embed_dim)

# If you want to just use tensors here, extract their weight data:
robot_embeddings = robot_embedding_matrix.weight.data  # shape (max_robot_id, embed_dim)
place_embeddings = place_embedding_matrix.weight.data  # shape (max_place_id, embed_dim)

In [ ]:
def sinusoidal_embedding(value, dim=32):
    """Sinusoidal embedding for a scalar value."""
    pe = torch.zeros(dim)
    position = torch.tensor(value, dtype=torch.float32)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(np.log(10000.0) / dim))
    pe[0::2] = torch.sin(position * div_term)
    pe[1::2] = torch.cos(position * div_term)
    return pe

def time_embedding(duration, max_duration):
    """Sinusoidal encoding for duration using 2D angle."""
    normalized = np.log(duration + 1) / np.log(max_duration + 1)
    angle = 2 * np.pi * normalized
    return torch.tensor([np.sin(angle), np.cos(angle)], dtype=torch.float32)

def label_embedding(y_start, y_end, num_labels=5):
    """Label transition encoding: -1 for origin labels, 1 for destination labels, 0 otherwise."""
    vec = torch.zeros(num_labels)
    for i in y_start:
        vec[i - 1] = -1
    for i in y_end:
        if vec[i - 1] == -1:
            vec[i - 1] = 0  # neutralize if label is both in start and end
        else:
            vec[i - 1] = 1
    return vec

def timestep_embedding(position, dim=32):
    """Sinusoidal positional encoding for timestep index."""
    pe = torch.zeros(dim)
    position = torch.tensor(position, dtype=torch.float32)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(np.log(10000.0) / dim))
    pe[0::2] = torch.sin(position * div_term)
    pe[1::2] = torch.cos(position * div_term)
    return pe

In [ ]:
def encode_actions(parsed_data, robot_embeddings, place_embeddings, timestep_embeddings, num_labels=5, embed_dim=32):
    max_duration = max(
        action['duration']
        for trajectory in parsed_data
        for timestep in trajectory
        for action in timestep
    )
    
    encoded_vectors = []

    for trajectory in parsed_data:
        encoded_trajectory = []

        for t_idx, timestep in enumerate(trajectory):
            timestep_vec = timestep_embeddings[t_idx]

            encoded_timestep = []

            for action in timestep:
                # Instead of sinusoidal, do lookup here:
                robot_vec = robot_embeddings[action['robot_id']]  # action['robot_id'] should be int index
                p_start_vec = place_embeddings[action['p_start']]
                p_end_vec = place_embeddings[action['p_end']]

                label_vec = label_embedding(action['y_start'], action['y_end'], num_labels=num_labels)
                time_vec = time_embedding(action['duration'], max_duration)

                full_vec = torch.cat([
                    robot_vec,
                    p_start_vec,
                    p_end_vec,
                    label_vec,
                    time_vec,
                    timestep_vec
                ])

                encoded_timestep.append(full_vec)
            encoded_trajectory.append(encoded_timestep)

        encoded_vectors.append(encoded_trajectory)

    return encoded_vectors

In [ ]:
# Parse the data from the log file
parsed_data = parse_trajectories("Datasets/Dataset1/BigDataset.txt")

# Set the number of labels manually
num_labels = 7  # Change this based on the number of labels you want

# Example: create embedding matrices (trainable parameters should be managed separately during training)
robot_embedding_matrix = nn.Embedding(max_robot_id, embed_dim)
place_embedding_matrix = nn.Embedding(max_place_id, embed_dim)

# Pass the weight tensors:
encoded_vectors = encode_actions(
    parsed_data,
    robot_embedding_matrix.weight.data,
    place_embedding_matrix.weight.data,
    timestep_embeddings=timestep_embedding_matrix.weight.data,
    num_labels=num_labels,
    embed_dim=embed_dim
)

# Print the encoded vector for the first timestep in the first trajectory
print(encoded_vectors[0][0][0])  # First timestep in the first trajectory
print(f"Vector size: {encoded_vectors[0][0][0].shape}")  # Expected size

In [ ]:
import torch

def pad_trajectories(encoded_vectors):
    max_timesteps = max(len(traj) for traj in encoded_vectors)
    max_actions = max(len(ts) for traj in encoded_vectors for ts in traj)
    feature_dim = encoded_vectors[0][0][0].shape[-1]

    padded_trajectories = []
    action_masks = []
    timestep_masks = []

    for traj in encoded_vectors:
        padded_traj = []
        traj_action_mask = []
        for ts in traj:
            # Pad actions within this timestep
            real_actions = len(ts)
            padded_ts = ts + [torch.zeros(feature_dim)] * (max_actions - real_actions)
            padded_ts_tensor = torch.stack(padded_ts)  # (A, D)
            padded_traj.append(padded_ts_tensor)

            # Action mask: 1 for real actions, 0 for padded
            action_mask = [1] * real_actions + [0] * (max_actions - real_actions)
            traj_action_mask.append(torch.tensor(action_mask, dtype=torch.bool))  # (A,)

        # Pad timesteps
        num_real_timesteps = len(traj)
        empty_ts = torch.zeros(max_actions, feature_dim)
        padded_traj += [empty_ts] * (max_timesteps - num_real_timesteps)
        padded_traj_tensor = torch.stack(padded_traj)  # (T, A, D)
        padded_trajectories.append(padded_traj_tensor)

        # Pad action mask
        empty_action_mask = torch.zeros(max_actions, dtype=torch.bool)
        traj_action_mask += [empty_action_mask] * (max_timesteps - num_real_timesteps)
        action_masks.append(torch.stack(traj_action_mask))  # (T, A)

        # Timestep mask: 1 for real timesteps, 0 for padded
        timestep_mask = [1] * num_real_timesteps + [0] * (max_timesteps - num_real_timesteps)
        timestep_masks.append(torch.tensor(timestep_mask, dtype=torch.bool))  # (T,)

    return (
        torch.stack(padded_trajectories),  # (B, T, A, D)
        torch.stack(action_masks),         # (B, T, A)
        torch.stack(timestep_masks),       # (B, T)
    )


data_tensor, action_mask, timestep_mask = pad_trajectories(encoded_vectors)

In [ ]:
print("Data Tensor Shape:", data_tensor.shape)          # (B, T, A, D)
print("Action Mask Shape:", action_mask.shape)          # (B, T, A)
print("Timestep Mask Shape:", timestep_mask.shape)      # (B, T)

# --- Check a specific trajectory ---
i = 0  # Check the first trajectory

print(f"\nTrajectory {i}:")
print("Timestep lengths:", [sum(am).item() for am in action_mask[i]])  # Number of real actions per timestep
print("Timestep mask:", timestep_mask[i].int().tolist())               # 1 if real timestep, 0 if padded

# --- Visual check for one timestep ---
t = 2  # First timestep
print(f"\nTrajectory {i}, Timestep {t} — Action Mask:", action_mask[i][t].int().tolist())
print(f"Data tensor (values) shape: {data_tensor[i][t].shape}")
print("First action vector (real or zero):", data_tensor[i][t][0])  # Show first 5 dims of first action

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

# Assuming you already have these:
# data_tensor: shape (B, T, A, D)
# action_mask: shape (B, T, A)
# timestep_mask: shape (B, T)

# 1. Create a TensorDataset
dataset = TensorDataset(data_tensor, action_mask, timestep_mask)

# 2. Define split sizes (e.g., 70% train, 15% val, 15% test)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# 3. Random split
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42)
)

# 4. Create DataLoaders
batch_size = 16  # or any other suitable size

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
print("Train batches:", len(train_loader))
print("Validation batches:", len(val_loader))
print("Test batches:", len(test_loader))

# Check one batch
for data_batch, action_mask_batch, timestep_mask_batch in train_loader:
    print("Data batch shape:", data_batch.shape)  # (B, T, A, D)
    print("Action mask shape:", action_mask_batch.shape)  # (B, T, A)
    print("Timestep mask shape:", timestep_mask_batch.shape)  # (B, T)
    break
